In [1]:
import pickle
import wikipediaapi

import nltk
from nltk.corpus import stopwords
import wikipediaapi
from nltk.tokenize import RegexpTokenizer
import re
from bs4 import BeautifulSoup
import re

from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

ps = nltk.stem.porter.PorterStemmer()

In [2]:
import tensorflow_hub as hub
import tensorflow_text
import tensorflow as tf

In [3]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.neighbors import KNeighborsClassifier

In [4]:
section = "history"

In [5]:
with open('../Instance Classification/section_dict.pkl', 'rb') as f:
    section_dict = pickle.load(f)

## Creating Training Data

In [6]:
wiki_wiki = wikipediaapi.Wikipedia('en', extract_format = wikipediaapi.ExtractFormat.HTML)
section_titles, related_keywords = section_dict[ps.stem(section)]

In [7]:
data_raw = []
documents = []
itr = [0]

def get_data(data_raw, documents, kw, section, itr):

    try:
    
        page = wiki_wiki.page(kw)
        kw_data = []

        for section_ in page.sections:
            title, content = section_.title, section_.text
            soup = BeautifulSoup(content, "html.parser")

            for p in soup.findAll("p"):
                p_text = p.text
                kw_data.append((kw, p_text, (ps.stem(title) == ps.stem(section))))

        data_raw.extend(kw_data)

    except:
        
        get_data(data_raw, documents, kw, section, itr)

for i, kw in enumerate(related_keywords):
    
    if i % 100 == 0:
        print(i, kw)

    get_data(data_raw, documents, kw, section, itr)

0 system dynamics
100 agile method
200 teaching
300 cache line
400 compositionality
500 attribute grammar
600 outsourcing
700 gamification


In [8]:
df = pd.DataFrame(data_raw, columns = ["keyword", "sentences", "label"])

In [9]:
data = pd.DataFrame(columns = df.columns)
for kw in related_keywords:
    df_subset = df[df["keyword"] == kw]
    
    pos_data = df_subset[df_subset["label"] != False]
    neg_data = df_subset[df_subset["label"] == False]
    
    num_pos, num_neg = len(pos_data), len(neg_data)
    
    if num_pos >= num_neg:
        data = pd.concat((data, pos_data, neg_data), axis = 0)
    else:
        data = pd.concat((data, pos_data, neg_data.sample(n = num_pos)), axis = 0)

## Training ALBERT

In [10]:
def clean_text(text):
    cleaned_text = re.sub(r'[^A-Za-z0-9 ]+', '', text)
    return cleaned_text.lower()
    
def clean_instances(data):
    instances = data["sentences"]
    cleaned_instances = [clean_text(inst) for inst in instances]
    
    df_new = pd.DataFrame(columns = data.columns)
    df_new[data.columns] = data[data.columns]
    df_new["sentences"] = cleaned_instances
    
    return df_new
    

data_clean = clean_instances(data)

train_text, test_text, train_labels, test_labels = train_test_split(data_clean["sentences"], data_clean["label"], test_size = 0.2)

train_text, test_text, np.array(train_text), np.array(test_text)
train_labels, test_labels = np.array(train_labels), np.array(test_labels)

In [11]:
albert_url='https://tfhub.dev/tensorflow/albert_en_base/2'
encoder = hub.KerasLayer(albert_url)

preprocessor_url="https://tfhub.dev/tensorflow/albert_en_preprocess/3"
preprocessor = hub.KerasLayer(preprocessor_url)

In [12]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
encoder_inputs = preprocessor(text_input)

In [13]:
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]     
sequence_output = outputs["sequence_output"]
pooled_output

<KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'keras_layer')>

In [14]:
embedding_model = tf.keras.Model(text_input, pooled_output)

In [15]:
model = tf.keras.Sequential()
model.add(embedding_model)

model.add(tf.keras.layers.Dense(30, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Dense(1))
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [16]:
model.fit(train_text, train_labels.astype(int), validation_data = (test_text, test_labels.astype(int)), epochs = 4)

Epoch 1/4
126/126 [==============================] - 31528s 252s/step - loss: 0.6816 - accuracy: 0.5568 - val_loss: 0.6690 - val_accuracy: 0.4975
Epoch 2/4
126/126 [==============================] - 466s 4s/step - loss: 0.5969 - accuracy: 0.6462 - val_loss: 0.6194 - val_accuracy: 0.6683
Epoch 3/4
126/126 [==============================] - 467s 4s/step - loss: 0.5576 - accuracy: 0.7004 - val_loss: 0.5695 - val_accuracy: 0.7309
Epoch 4/4
126/126 [==============================] - 3568s 29s/step - loss: 0.5221 - accuracy: 0.7354 - val_loss: 0.5289 - val_accuracy: 0.7597
